In [ ]:
import cv2
from PIL import Image
from google.colab.patches import cv2_imshow
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from PIL import Image, ImageCms
from skimage import feature
from sklearn.cluster import MiniBatchKMeans
from skimage.util import random_noise
import glob



In [ ]:
image_paths=glob.glob("/content/images/*")
im=Image.open(image_paths[0])
ratio=1
im=im.resize([int(im.size[i]/ratio) for i in range(len(im.size))])
#im=im.rotate(90)
im_gray = im.convert('L')
im_gray_array = np.asarray(im_gray,dtype=np.float)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
im_gray_array = clahe.apply(im_gray_array.astype("uint8"))
X=im_gray_array.reshape((-1, 1))
Image.fromarray(im_gray_array)

In [ ]:
def valueHistEq(img):
  if str(type(img))=="<class 'PIL.Image.Image'>":
    img=np.asarray(img)  
  HSV=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
  HSV[:, :, 2] = cv2.equalizeHist(HSV[:, :, 2])
  BGR=cv2.cvtColor(HSV,cv2.COLOR_HSV2BGR,img)
  return BGR

def clipValueHistEq(img,clipLimit=2.0,girdSize=(8,8)):
  if str(type(img))=="<class 'PIL.Image.Image'>":
    img=np.asarray(img) 
  HSV=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
  HSV[:, :, 2] = cv2.equalizeHist(HSV[:, :, 2])
  BGR=cv2.cvtColor(HSV,cv2.COLOR_HSV2BGR,img)
  return BGR

def yHistEq(img):
  if str(type(img))=="<class 'PIL.Image.Image'>":
    img=np.asarray(img) 
  ycrcb=cv2.cvtColor(img,cv2.COLOR_BGR2YCR_CB)
  ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])
  BGR=cv2.cvtColor(ycrcb,cv2.COLOR_YCR_CB2BGR,img)
  return BGR


def clipYHistEq(img,clipLimit=2.0,girdSize=(8,8)):
  if str(type(img))=="<class 'PIL.Image.Image'>":
    img=np.asarray(img) 
  ycrcb=cv2.cvtColor(img,cv2.COLOR_BGR2YCR_CB)
  clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
  ycrcb[:, :, 0] = clahe.apply(ycrcb[:, :, 0].astype("uint8"))
  BGR=cv2.cvtColor(ycrcb,cv2.COLOR_YCR_CB2BGR,img)
  return BGR

def get_colorized_edges(img,sigma=2):
  img_gray_array=np.asarray(img.convert('L'))
  edges=feature.canny(img_gray_array,sigma=sigma).astype("uint8")
  kernel=np.ones((3,3))
  edges=cv2.dilate(edges,kernel,iterations=1)
  edged_im1=cv2.bitwise_and(np.asarray(img),np.asarray(img),mask=edges)
  return edged_im1.astype("uint8")

def draw_edges_over_image(img,sigma=2,color=(0,0,0),offset=0):
  if str(type(img))=="<class 'PIL.Image.Image'>":
    img_gray_array=np.asarray(img.convert('L'))
  elif str(type(img))=="<class 'numpy.ndarray'>":
    img_gray_array=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  else:
    print ("img type {} not supported".format(type(img)))
  edges=feature.canny(img_gray_array,sigma=sigma).astype("uint8")
  kernel=np.ones((3,3))
  #edges=cv2.dilate(edges,kernel,iterations=1)
  edges=np.roll(edges,offset,axis=1)
  edges_rgb=cv2.bitwise_and(np.ones(np.array(img).shape)*127,
                            np.ones(np.array(img).shape)*127,
                            mask=edges).astype("uint8")
  edges_rgb=np.where(edges_rgb==(127,127,127),color,(127,127,127))
  edged_im2=np.where(edges_rgb!=(127,127,127),edges_rgb,np.array(img))
  return edged_im2.astype("uint8")

def reduce_color_space(img,ncolors=10):
  imlab = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2LAB)
  (h, w) = imlab.shape[:2]
  imlab = imlab.reshape((imlab.shape[0] * imlab.shape[1], 3))
  clt = MiniBatchKMeans(n_clusters =ncolors)
  labels = clt.fit_predict(imlab)
  quant = clt.cluster_centers_.astype("uint8")[labels]
  quant = quant.reshape((h, w, 3))
  return cv2.cvtColor(quant, cv2.COLOR_LAB2BGR)

def shift_rgb(img,offset=10):
  if str(type(img))=="<class 'PIL.Image.Image'>":
    img=np.asarray(img) 
  img=img.copy()
  for channel in range(img.shape[2]):
    img[:,:,channel]=np.roll(img[:,:,channel],offset*channel,axis=1)
  return img

In [ ]:
Image.fromarray(shift_rgb(im,15))

In [ ]:
noise_img = random_noise(np.asarray(im), mode='s&p',amount=0.05)
noise_img = np.array(255*noise_img, dtype = 'uint8')
Image.fromarray(draw_edges_over_image(reduce_color_space(clipValueHistEq(im),3),
                                      color=(255,255,255),
                                      offset=0,
                                      sigma=4))

In [ ]:
Image.fromarray(draw_edges_over_image(reduce_color_space(im,6)))

In [ ]:
 Image.fromarray(get_colorized_edges(im))

In [ ]:
if np.asarray(im).shape[0]*16/9<np.asarray(im).shape[0]:
  stack=np.vstack((np.asarray(im),
                   yHistEq(np.asarray(im)),
                   valueHistEq(np.asarray(im)),
                   clipYHistEq(np.asarray(im)),
                   clipValueHistEq(np.asarray(im))))
else:
  stack=np.hstack((np.asarray(im),
                   yHistEq(np.asarray(im)),
                   valueHistEq(np.asarray(im)),
                   clipYHistEq(np.asarray(im)),
                   clipValueHistEq(np.asarray(im))))
Image.fromarray(stack.astype("uint8")).save("/content/collage.png")
Image.fromarray(stack.astype("uint8"))

In [ ]:
cv2.imwrite("/content/collage.png",np.vstack((clipYHistEq(np.asarray(im)),np.asarray(im)))[:,:,::-1])

In [ ]:
stack=np.hstack((np.asarray(im),
                 clipValueHistEq(clipYHistEq((np.asarray(im)))),
                 clipYHistEq(clipValueHistEq((np.asarray(im))))))
Image.fromarray(stack.astype("uint8"))

# kmeans y tal

In [ ]:
Image.fromarray(stack.astype("uint8")).save("/content/collage.png")

In [ ]:
srgb_profile = ImageCms.createProfile("sRGB")
lab_profile  = ImageCms.createProfile("LAB")

rgb2lab_transform = ImageCms.buildTransformFromOpenProfiles(srgb_profile, lab_profile, "RGB", "LAB")
eq_im=Image.fromarray(hisEqulColor(np.asarray(im)))
lab_im = ImageCms.applyTransform(eq_im, rgb2lab_transform)

In [ ]:
n_clusters=5
colores=np.random.randint(0,255,(n_clusters,3))
k_means = KMeans(n_clusters=n_clusters)
k_means.fit(X)
etiquetas = k_means.labels_
im_kmeans_1=np.array([colores[etiqueta] for etiqueta in etiquetas])
im_kmeans_1=im_kmeans_1.reshape((im.size[1],im.size[0],3))

if im_compressed.shape[0]>im_compressed.shape[0]:
  stack=np.hstack((im_kmeans_1,np.asarray(im)))
else:
  stack=np.vstack((im_kmeans_1,np.asarray(im)))
Image.fromarray(stack.astype("uint8"))

In [ ]:
k_means = KMeans(n_clusters=n_clusters)
X_2=np.asarray(im).reshape(im.size[0]*im.size[1],3)
k_means.fit(X_2)
etiquetas = k_means.labels_
im_kmeans_2=np.array([colores[etiqueta] for etiqueta in etiquetas])
im_kmeans_2=im_kmeans_2.reshape((im.size[1],im.size[0],3))

if im_compressed.shape[0]>im_compressed.shape[0]:
  stack=np.hstack((im_kmeans_2,np.asarray(im)))
else:
  stack=np.vstack((im_kmeans_2,np.asarray(im)))
Image.fromarray(stack.astype("uint8"))

In [ ]:
k_means = KMeans(n_clusters=n_clusters)
X_3=np.asarray(lab_im).reshape(lab_im.size[1]*lab_im.size[0],3)
k_means.fit(X_3)
etiquetas = k_means.labels_
im_kmeans_3=np.array([colores[etiqueta] for etiqueta in etiquetas])
im_kmeans_3=im_kmeans_3.reshape((im.size[1],im.size[0],3))

if im_kmeans_3.shape[0]>im_kmeans_3.shape[0]:
  stack=np.hstack((im_kmeans_3,np.asarray(im)))
else:
  stack=np.vstack((im_kmeans_3,np.asarray(im)))
Image.fromarray(stack.astype("uint8"))

In [ ]:
collage=np.hstack((im_kmeans_1,im_kmeans_2,im_kmeans_3,im))
Image.fromarray(collage.astype("uint8"))